### Extraindo informações mais gerais dos produtos na categoria Saúde e Bem-Estar:
* Título
* Marca
* Preço com Desconto
* Preço sem desconto

In [31]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup


chrome_options = Options()
chrome_options.add_argument("--headless") 


urls = [
    
    "https://drogariasoares.com.br/categoria/acessorios/51",
    "https://drogariasoares.com.br/categoria/promocao-de-carnaval/173?pagina=3",
    "https://drogariasoares.com.br/categoria/alimentos-e-bebidas/52?pagina=3",
    "https://drogariasoares.com.br/categoria/dentadura/53",
    "https://drogariasoares.com.br/categoria/fitoterapicos-e-homeopaticos/18",
    "https://drogariasoares.com.br/categoria/geriatria/55",
    "https://drogariasoares.com.br/categoria/lentes-e-acessorios/56",
    "https://drogariasoares.com.br/categoria/nutricosmeticos/58",
    "https://drogariasoares.com.br/categoria/ortopedia/59",
    "https://drogariasoares.com.br/categoria/preservativos-e-lubrificantes/60?pagina=2",
    "https://drogariasoares.com.br/categoria/produtos-naturais/61?pagina=2"
    
]


all_products_data = []


for url in urls:
    
    driver = webdriver.Chrome(options=chrome_options)
    driver.get(url)
    time.sleep(3)  

    
    html_content = driver.page_source

   
    driver.quit()

    
    soup = BeautifulSoup(html_content, "html.parser")

    
    produtos = soup.find_all("div", class_="item-prod")

    
    for produto in produtos:
        titulo = produto.find("h2").text.strip()
        marca = produto.find("small", class_="marca").text.strip()
        preco_sem_desconto_element = produto.find("li", class_="preco text-center").find("p", class_='preco-de')
        preco_sem_desconto = preco_sem_desconto_element.text.strip() if preco_sem_desconto_element else ""
        preco_com_desconto = produto.find("p", class_="preco-por").strong.span.text.strip()

       
        product_data = {
            "Título": titulo,
            "Marca": marca,
            "Preço sem Desconto": preco_sem_desconto,
            "Preço com Desconto": preco_com_desconto
        }
        all_products_data.append(product_data)


df1 = pd.DataFrame(all_products_data)


df1.drop_duplicates(inplace=True)



In [32]:
df1

,Título,Marca,Preço sem Desconto,Preço com Desconto
0,AGULHA 30X8 UNID,BD HEALTH CARE,,"R$ 0,50"
1,AGULHA 30X7 BD UNID,BD HEALTH CARE,,"R$ 0,50"
2,COLETOR UNIVERSAL 80ML,GPZ - GPZ COMERCIAL LTDA,,"R$ 1,50"
3,SERINGA 10ML S/AGU BD PLASTI LISA,BD HEALTH CARE,,"R$ 3,99"
4,"AGULHA 13X4,5 UNID.",BD HEALTH CARE,,"R$ 0,50"
...,...,...,...,...
359,OLEO DE LINHACA 1000MGR C/60 CAPS SCHRAI,SCHRAIBER,,"R$ 38,10"
360,PROPOMAX EXTRATO PROPOLIS C/30 CAPS,APIS FLORA,,"R$ 35,40"
361,PROPOMAX NOITE 140ML,APIS FLORA,,"R$ 36,12"
362,SUNDOWN NAT TRIPLE OMEGA C/120CAPS,SUNDOWN NESTLE,,"R$ 239,00"


### Extraindo os códigos dos produtos na categoria Saúde e Bem-Estar:
* ID
* SKU
##### OBS: Infelizmente, no site da Drogaria Soares não foi localizado o código relativo ao EAN, por esse motivo, o mesmo não foi apresentado nesse código.

In [33]:
import concurrent.futures
import re  # Adicionada a importação do módulo re (Expressões Regulares)
import requests
from pyquery import PyQuery as pq
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
import pandas as pd
import time

def extrair_links_produtos():
    urls = [
        
    "https://drogariasoares.com.br/categoria/acessorios/51",
    "https://drogariasoares.com.br/categoria/promocao-de-carnaval/173?pagina=3",
    "https://drogariasoares.com.br/categoria/alimentos-e-bebidas/52?pagina=3",
    "https://drogariasoares.com.br/categoria/dentadura/53",
    "https://drogariasoares.com.br/categoria/fitoterapicos-e-homeopaticos/18",
    "https://drogariasoares.com.br/categoria/geriatria/55",
    "https://drogariasoares.com.br/categoria/lentes-e-acessorios/56",
    "https://drogariasoares.com.br/categoria/nutricosmeticos/58",
    "https://drogariasoares.com.br/categoria/ortopedia/59",
    "https://drogariasoares.com.br/categoria/preservativos-e-lubrificantes/60?pagina=2",
    "https://drogariasoares.com.br/categoria/produtos-naturais/61?pagina=2"
]
    
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36'
    }
    links = []
    
    for url in urls:
        response = requests.get(url, headers=headers)
        
        if response.status_code == 200:
            doc = pq(response.text)
            produtos = doc('.link-prod')
            
            for produto in produtos:
                href = pq(produto).attr('href')
                if 'a-4-body-lotion-maca-220g' not in href:
                    links.append(href)

    return links

def extrair_info_produto(url):
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=options)

    driver.get(url)
    time.sleep(5)

    page_source = driver.page_source

    doc = pq(page_source)
    
    script_content = doc('script').text()  # Obtém o conteúdo dos scripts na página

    # Usando expressão regular para encontrar o ID e o SKU nos scripts
    id_match = re.search(r"id:'(\d+)'", script_content)
    sku_match = re.search(r"sku:'(\d+)'", script_content)
    

    if id_match and sku_match:
        id_produto = id_match.group(1)
        sku_produto = sku_match.group(1)
        return id_produto, sku_produto
    else:
        return None, None

def extrair_info_produto_wrapper(url):
    try:
        return extrair_info_produto(url)
    except Exception as e:
        print(f"Erro ao extrair informações de {url}: {e}")
        return None, None

links_produtos = extrair_links_produtos()

dados_produtos = []
with concurrent.futures.ThreadPoolExecutor(max_workers=8) as executor:
    resultados = executor.map(extrair_info_produto_wrapper, ['https://drogariasoares.com.br' + link for link in links_produtos])
    for resultado in resultados:
        if resultado[0] is not None and resultado[1] is not None:
            dados_produtos.append(resultado)

df2 = pd.DataFrame(dados_produtos, columns=['ID do Produto', 'SKU'])
df2.drop_duplicates(inplace=True)



Erro ao extrair informações de https://drogariasoares.com.br/esmalte-dailus-queridinhos-pipoca-doce-8ml/105972-01: Message: timeout: Timed out receiving message from renderer: 298.759
  (Session info: chrome-headless-shell=122.0.6261.129)
Stacktrace:
	GetHandleVerifier [0x00538D03+51395]
	(No symbol) [0x004A5F61]
	(No symbol) [0x0035E13A]
	(No symbol) [0x0034EE71]
	(No symbol) [0x0034ECA3]
	(No symbol) [0x0034D98B]
	(No symbol) [0x0034E1CD]
	(No symbol) [0x00358E6A]
	(No symbol) [0x00367101]
	(No symbol) [0x0036A976]
	(No symbol) [0x0034E5F8]
	(No symbol) [0x00366F9F]
	(No symbol) [0x003CA87B]
	(No symbol) [0x003B3C26]
	(No symbol) [0x0038C629]
	(No symbol) [0x0038D40D]
	GetHandleVerifier [0x008B68D3+3712147]
	GetHandleVerifier [0x008F5CBA+3971194]
	GetHandleVerifier [0x008F0FA8+3951464]
	GetHandleVerifier [0x005E9D09+776393]
	(No symbol) [0x004B1734]
	(No symbol) [0x004AC618]
	(No symbol) [0x004AC7C9]
	(No symbol) [0x0049DDF0]
	BaseThreadInitThunk [0x76A07BA9+25]
	RtlInitializeExcepti

In [34]:
df2

,ID do Produto,SKU
0,1638426,66665
4,1638466,66663
8,8888841,68952
12,1638472,88512
16,1638421,66659
...,...,...
1434,4690701,86102
1438,5842803,98001
1442,9990063,102737
1446,3570613,89211


### Unificando e Tratando os Dataframes:

In [35]:
reindexed_df_final = df2.set_index(pd.Index(range(364)))

In [36]:
reindexed_df_final

,ID do Produto,SKU
0,1638426,66665
1,1638466,66663
2,8888841,68952
3,1638472,88512
4,1638421,66659
...,...,...
359,4690701,86102
360,5842803,98001
361,9990063,102737
362,3570613,89211


In [37]:
df1["ID"] = reindexed_df_final["ID do Produto"]
df1["SKU"] = reindexed_df_final["SKU"]

In [38]:
df1

,Título,Marca,Preço sem Desconto,Preço com Desconto,ID,SKU
0,AGULHA 30X8 UNID,BD HEALTH CARE,,"R$ 0,50",1638426,66665
1,AGULHA 30X7 BD UNID,BD HEALTH CARE,,"R$ 0,50",1638466,66663
2,COLETOR UNIVERSAL 80ML,GPZ - GPZ COMERCIAL LTDA,,"R$ 1,50",8888841,68952
3,SERINGA 10ML S/AGU BD PLASTI LISA,BD HEALTH CARE,,"R$ 3,99",1638472,88512
4,"AGULHA 13X4,5 UNID.",BD HEALTH CARE,,"R$ 0,50",1638421,66659
...,...,...,...,...,...,...
359,OLEO DE LINHACA 1000MGR C/60 CAPS SCHRAI,SCHRAIBER,,"R$ 38,10",4690701,86102
360,PROPOMAX EXTRATO PROPOLIS C/30 CAPS,APIS FLORA,,"R$ 35,40",5842803,98001
361,PROPOMAX NOITE 140ML,APIS FLORA,,"R$ 36,12",9990063,102737
362,SUNDOWN NAT TRIPLE OMEGA C/120CAPS,SUNDOWN NESTLE,,"R$ 239,00",3570613,89211


In [39]:
df1.to_csv('Saúde_e_Bem_Estar_Soaeres.csv')